# Fake News Detection

The main porpose of this model is to classify the differrence between fake news and real news.

### Importing libraries

In [30]:
import pandas as pd
import re
import string
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [31]:
df_raw_true = pd.read_csv(r".\database\True.csv", encoding = 'utf-8')

In [32]:
df_raw_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [33]:
df_raw_fake = pd.read_csv(r".\database\Fake.csv", encoding = 'utf-8')

In [34]:
df_raw_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [35]:
df_raw_true['label'] = 1                            #1 for real news

In [36]:
df_raw_fake['label'] = 0                             #0 for fake news 

In [37]:
df_true = df_raw_true[['text', 'label']]

In [38]:
df_fake = df_raw_fake[['text', 'label']]

In [39]:
df = pd.concat([df_true, df_fake])                #merges both the dataframes

In [40]:
df.sample(5)

,text,label
17110,ANKARA (Reuters) - Iranian Supreme Leader Ayat...,1
20734,CHISINAU (Reuters) - The Moldovan government s...,1
13578,Paul Joseph Watson is the bomb! He makes the g...,0
14759,BEIRUT (Reuters) - A member of the political p...,1
18000,Wilbur Ross spoke with WaPo about true tax r...,0


In [41]:
df.shape

(44898, 2)

### Cleaning database

In [42]:
df['text'] = df['text'].str.lower()     #Changing every character to lower case 

In [43]:
#Makes a dictionary containing all punctuations with value 0
punct_dict = dict((ord(punct), None) for punct in string.punctuation)

for punc in punct_dict:
    df['text'] = df['text'].replace(punc, ' ')    

In [44]:
df.sample(5)

,text,label
19607,,0
11655,"last november, the unc campus was full of angr...",0
5909,(reuters) - the u.s. department of homeland se...,1
3478,washington (reuters) - liberal activists are u...,1
549,donald trump s weak response to the racially m...,0


In [45]:
def extract_words(sentence):
    ignore_words = ['a', 'the', 'if', 'br', 'and', 'of', 'to', 'is']
    words = re.sub("[^\w]", " ",  sentence).split()
    
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned 


## Training Model

In [46]:
x = df['text']
y = df['label']

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle = True)         #Splitting the data

In [48]:
vec = TfidfVectorizer(stop_words = 'english')

In [49]:
xv_train = vec.fit_transform(x_train)
xv_test = vec.transform(x_test)

In [50]:
logreg = LogisticRegression()
logreg.fit(xv_train, y_train)

LogisticRegression()

In [51]:
y_pred = logreg.predict(xv_test)

In [52]:
score = accuracy_score(y_pred, y_test)
print(f'Accuracy Score: {score}')

Accuracy Score: 0.9847661469933184


In [53]:
confusion_matrix(y_test, y_pred)

array([[5822,  111],
       [  60, 5232]], dtype=int64)

In [54]:
def news_predictor(news):
    vec_news = vec.transform([news])
    
    pred = logreg.predict(vec_news)
    label = {1:'Real', 0:'Fake'}
    return label[int(pred)]

In [ ]:
#news_predictor(<string>) gives output 'Real' or 'Fake'